In [ ]:
import gurobipy as gp
from gurobipy import GRB

env = gp.Env(empty=True)
env.setParam("OutputFlag", 0)
env.start()

m = gp.Model(env=env)

def schedule_model(E,T,R,S, capacities, class_sizes):
    """
    Inputs:
    E - list of exams to be scheduled
    T - list of timeslots available
    R - list of rooms available
    S - dictionary of student schedules
    capacities - dictionary of room capacities
    class_sizes - dictionary mapping exam to number of students

    """
    size_e = len(E)
    size_t = len(T)
    size_r = len(R)

    y_vars = [(e, t) for e in range(size_e) for t in range(size_t)]
    z_vars = [(e_1, e_2, t) for e_2 in range(size_e) for e_1 in range(size_e) for t in range(size_t) if e_2 < e_1]
    x = m.addVars(range(size_e), range(size_t), range(size_r),vtype=GRB.BINARY)

    y = m.addVars(y_vars, vtype=GRB.BINARY)
    z = m.addVars(z_vars,vtype=GRB.CONTINUOUS)
    w = [1]*size_t # figure out late penalties late
    #assume only 1 student overlap between every pair of exams
    s = {(e_1, e_2): 1 for e_1 in range(size_e) for e_2 in range(size_e) if e_2 < e_1}
    lamb = 1
    m.setObjective(gp.quicksum(w[j]*x[i, j, k] for i in range(size_e) for j in range(size_t) for k in range(size_r))+ 
                   lamb*gp.quicksum(s[(e_1, e_2)]*z[e_1, e_2, t] for e_1 in range(size_e) for e_2 in range(size_e) for t in range(size_t) if e_2 < e_1),
                GRB.MINIMIZE)
    
    # CONSTRAINTS:
    m.addConstrs((gp.quicksum(x[e,t,r] for t in range(size_t) for r in range(size_r)) == 1 for e in range(size_e)), name="assign_once")

    m.addConstrs((gp.quicksum(class_sizes[e]*x[e,t,r] for e in range(size_e)) <= capacities[r] for t in range(size_t) for r in range(size_r)), name="capacity")

    m.addConstrs((y[e,t] == gp.quicksum(x[e,t,r] for r in range(size_r)) for e in range(size_e) for t in range(size_t)), name="y_def")

    m.addConstrs(z[e_1,e_2,t] <= y[e_1,t] for t in range(size_t) for e_1, e_2 in range(size_e) if e_1 < e_2)
    m.addConstrs(z[e_1,e_2,t] <= y[e_2,t] for t in range(size_t) for e_1, e_2 in range(size_e) if e_1 < e_2)
    m.addConstrs(z[e_1,e_2,t] >= y[e_1,t] + y[e_2,t] - 1 for t in range(size_t) for e_1, e_2 in range(size_e) if e_1 < e_2)









m.addConstr(12000 + l + 10000 + mmb1 >= 40000)


m.optimize()

print("The optimal solution is {}".format(m.x))
print("The total cost is {}".format(m.ObjVal)